In [ ]:
using PerlaTonettiWaugh, Parameters

### Baseline Steady State

In [ ]:
d_baseline = 3.0426 # d_0
params = parameter_defaults(γ = 1)
settings = settings_defaults();
params_baseline = merge(params, (d = d_baseline,));

In [ ]:
stationary_baseline = stationary_algebraic(params_baseline, settings);
println("g: $(stationary_baseline.g)")
println("z_hat: $(stationary_baseline.z_hat)")
println("Omega: $(stationary_baseline.Ω)")
println("c: $(stationary_baseline.c)")
println("U_bar: $(stationary_baseline.U_bar)")
println("lambda: $(stationary_baseline.λ_ii)")
println("y: $(stationary_baseline.y)");

### Counterfactual Steady State

In [ ]:
d_counterfactual = 1 + 0.99*(d_baseline - 1);
params_counterfactual = merge(params, (d = d_counterfactual,))
stationary_counterfactual = stationary_algebraic(params_counterfactual, settings);

In [ ]:
println("g: $(stationary_counterfactual.g)")
println("z_hat: $(stationary_counterfactual.z_hat)")
println("Omega: $(stationary_counterfactual.Ω)")
println("c: $(stationary_counterfactual.c)")
println("U_bar: $(stationary_counterfactual.U_bar)")
println("lambda: $(stationary_counterfactual.λ_ii)")
println("y: $(stationary_counterfactual.y)")
ACR_full = 100*(-1/params.θ)*log(stationary_counterfactual.λ_ii/stationary_baseline.λ_ii)
println("ACR_full: $(ACR_full)");
@show d_U_bar = stationary_counterfactual.U_bar - stationary_baseline.U_bar;

### Decomposition 1: $\frac{\partial c}{\partial d}$ (change in consumption when changing trade cost, holding fixed Omega, zhat, g)

In [ ]:
sol = steady_state_from_g(stationary_baseline.g, stationary_baseline.z_hat, stationary_baseline.Ω, params_counterfactual, settings);

In [ ]:
println("c: $(sol.c)")
println("z_hat: $(sol.z_hat)")
println("Omega: $(sol.Ω)")
println("lambda: $(sol.λ_ii)")
println("U_bar: $(sol.U_bar)")
U_bar_partial = sol.U_bar;
ACR_partial = 100*(-1/params.θ)*log(sol.λ_ii/stationary_baseline.λ_ii)
dc = sol.c - stationary_baseline.c
dc_frac = dc/ stationary_baseline.c
println("ACR_partial: $(ACR_partial)")
println("dc: $(dc)")
println("dc_frac: $(dc_frac)")
@show d_U_1 = 1/(stationary_baseline.c * params.ρ) # constant
@show d_U_bar - d_U_1  * dc
@show d_U_Comp1_frac=100*(d_U_bar-d_U_1*dc)/d_U_bar;

### Decomposition 2: $\frac{\partial c}{\partial g}$ (change in consumption when changing g, holding fixed Omega, zhat, d)

In [ ]:
sol = steady_state_from_g(1.01*stationary_baseline.g, stationary_baseline.z_hat, stationary_baseline.Ω, params_baseline, settings);

In [ ]:
println("c: $(sol.c)")
println("z_hat: $(sol.z_hat)")
println("Omega: $(sol.Ω)")
println("lambda: $(sol.λ_ii)")
println("U_bar: $(sol.U_bar)")
U_bar_partial = sol.U_bar;
ACR_partial = 100*(-1/params.θ)*log(sol.λ_ii/stationary_baseline.λ_ii)
println("ACR_partial: $(ACR_partial)")
@show dc_frac = (sol.c - stationary_baseline.c)/ stationary_baseline.c;

### Decomposition 3: $\frac{\partial g}{\partial d}$ (change in g when changing d, holding fixed Omega, zhat, and c)

In [ ]:
sol = steady_state_from_c(stationary_baseline.c, stationary_baseline.z_hat, stationary_baseline.Ω, params_counterfactual, settings);

In [ ]:
println("g: $(sol.g)")
println("c: $(sol.c)")
println("z_hat: $(sol.z_hat)")
println("Omega: $(sol.Ω)")
println("lambda: $(sol.λ_ii)")
println("U_bar: $(sol.U_bar)")
U_bar_partial = sol.U_bar;
ACR_partial = 100*(-1/params.θ)*log(sol.λ_ii/stationary_baseline.λ_ii)
println("ACR_partial: $(ACR_partial)")
@show dc_frac = (sol.c - stationary_baseline.c)/ stationary_baseline.c
@show dg_frac = (sol.g - stationary_baseline.g) / stationary_baseline.g;

### Decomposition 4: $\frac{\partial g}{\partial c}$ (change in g when changing c, holding fixed Omega, zhat, and d)

In [ ]:
sol = steady_state_from_c(stationary_baseline.c*1.01, stationary_baseline.z_hat, stationary_baseline.Ω, params_baseline, settings);

In [ ]:
println("g: $(sol.g)")
println("c: $(sol.c)")
println("z_hat: $(sol.z_hat)")
println("Omega: $(sol.Ω)")
println("lambda: $(sol.λ_ii)")
println("U_bar: $(sol.U_bar)")
U_bar_partial = sol.U_bar;
ACR_partial = 100*(-1/params.θ)*log(sol.λ_ii/stationary_baseline.λ_ii)
println("ACR_partial: $(ACR_partial)")
@show dc_frac = (sol.c - stationary_baseline.c)/ stationary_baseline.c
@show dg_frac = (sol.g - stationary_baseline.g) / stationary_baseline.g;

### Total Derivative

In [ ]:
D = total_derivative(params_baseline);

In [ ]:
@unpack U_1, U_2, ∂_c_d, ∂_c_g, ∂_g_c, ∂_g_d, d_U_d, total_decomp, check, planner_0, planner_0_frac, decomp_1_frac, decomp_2_frac, decomp_3_frac, decomp_4_frac = D;

In [ ]:
@show U_1
@show U_2
@show ∂_c_d
@show ∂_c_g
@show ∂_g_d
@show ∂_g_c
@show d_U_d
@show total_decomp
@show check
@show planner_0
@show planner_0_frac
@show decomp_1_frac
@show decomp_2_frac
@show decomp_3_frac
@show decomp_4_frac;